In [17]:
# Import some libraries

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

import os
import pickle
import numpy as np

from PIL import Image
from skimage import io

from matplotlib import pyplot as plt
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['figure.figsize'] = 15, 25

%matplotlib inline
%load_ext autoreload
%autoreload 2


import os
import random
import argparse

import numpy as np


from data_helper import LabeledDataset
from helper import compute_ats_bounding_boxes, compute_ts_road_map

from model_loader import get_transform, ModelLoader


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [21]:
image_folder = '/Users/nikhilvs/repos/nyu/vehicle-scene-generator/data'
annotation_csv = f'{image_folder}/annotation.csv'

labeled_scene_index = np.arange(120, 134)

labeled_trainset = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform(),
    extra_info=False
    )
dataloader = torch.utils.data.DataLoader(
    labeled_trainset,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )

In [23]:
model_loader = ModelLoader()

total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    # sample = sample.cuda()

    predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
    predicted_road_map = model_loader.get_binary_road_map(sample).cpu()

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes, target['bounding_box'][0])
    ts_road_map = compute_ts_road_map(predicted_road_map, road_image)

    total_ats_bounding_boxes += ats_bounding_boxes
    total_ts_road_map += ts_road_map

    # if opt.verbose:
    print(f'{i} - Bounding Box Score: {ats_bounding_boxes:.4} - Road Map Score: {ts_road_map:.4}')

print(f'{model_loader.team_name} - {model_loader.round_number} - Bounding Box Score: {total_ats_bounding_boxes / total:.4} - Road Map Score: {total_ts_road_map / total:.4}')



0 - Bounding Box Score: 0.0 - Road Map Score: 0.9307
1 - Bounding Box Score: 0.0 - Road Map Score: 0.9338
2 - Bounding Box Score: 0.0 - Road Map Score: 0.9343
3 - Bounding Box Score: 0.0 - Road Map Score: 0.9403


KeyboardInterrupt: 